# YouTube 熱門影片主題建模（BERTopic）Pipeline

以中文分析為核心，使用 `shibing624/text2vec-base-chinese` 產生向量嵌入，
並輸出主題結構（Hierarchy / Bubble）與 Sankey（Topic ↔ Search keyword）。

In [ ]:
need_transform_to_py = True

In [1]:
%load_ext autoreload
%autoreload 2
from path_setup import setup_project_root
root = setup_project_root()

import os, sys, jieba, hdbscan
import pandas as pd
from sentence_transformers import SentenceTransformer
from bertopic import BERTopic
from sklearn.feature_extraction.text import CountVectorizer
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Local modules
from etl_showcase.infrastructure.cleaning.data_preprocessor import (
    google_sheet_to_dataframe, 
    preprocess_dataframe,
)
from etl_showcase.infrastructure.cleaning.text_cleaner import (
    remove_all_punctuation
)
from etl_showcase.infrastructure.cleaning.text_tokenizer import (
    jieba_tokenizer
)
from etl_showcase.infrastructure.reporting.html_export import (
    save_plotly_html,
)
from etl_showcase.infrastructure.reporting.topic_visualizer import (
    visualize_topics_bubble, 
    visualize_hierarchical_clustering, 
    visualize_sankey, 
)
from etl_showcase.config.youtube import (
    YOUTUBE_SPREADSHEET_ID,
    YOUTUBE_SEARCH_VIDEOS_FUNCTION_NAME,
    VIDEO_COLUMN_ORDER,
)
from etl_showcase.infrastructure.datasource.google_sheets_api import (
    write_secret_json,
    delete_secret_json,
    get_full_google_sheet,
)

def data_source_text(keywords:str):
    return f"資料來源：每月月底從 YouTube 蒐集當月與關鍵字相關的前 300 筆影片標題與描述，並使用 BERTopic 進行主題分析，生成熱門議題。<br />本次搜尋關鍵字為：{keywords}。"

Using project root: C:\My data\0.change jobs\data_science_practice


In [2]:
youtube_videos = [[]]
write_secret_json()
try:
    youtube_videos = get_full_google_sheet(
        spreadsheet_id=YOUTUBE_SPREADSHEET_ID,
        sheet_name=YOUTUBE_SEARCH_VIDEOS_FUNCTION_NAME
    )
finally:
    delete_secret_json()
df_raw = google_sheet_to_dataframe(youtube_videos, VIDEO_COLUMN_ORDER)
print("Loaded rows:", len(df_raw))
df_raw.head()

Loaded rows: 1200


,Category,Search keyword,Video ID,Video title,Video description,Publish datetime,Channel name,Channel ID,Thumbnail URL
0,社會領域,社會議題,5B7AsWa2Mvo,Precht - KI vs Sozialsystem,社會保險#五險一金#勞資關係#社會結構#高齡化社會#人口紅利0:00 重點在這0:21 中國...,2025-08-28T08:30:05Z,寰宇新聞 頻道,UCp2f7tGJGN6R9Muxipem8Nw,https://i.ytimg.com/vi/5B7AsWa2Mvo/hqdefault.jpg
1,社會領域,社會議題,Abnm45OuQCA,いまや社会問題となっている、ある「タブー」について、話す時がきました。,社會保險#五險一金#勞資關係#社會結構#高齡化社會#人口紅利0:00 重點在這0:21 中國...,2025-08-28T09:30:14Z,寰宇全視界,UCiOR3zQCU-tLza5g1MuqABA,https://i.ytimg.com/vi/Abnm45OuQCA/hqdefault.jpg
2,社會領域,社會議題,zga_r5nfErc,無退路_不向腐敗低頭#社會議題 #台灣 #ai,詞：LowDon 曲：Suno 唱：Suno 錯誤的執行，錯誤的人選，傷害著社會，失去著民心...,2025-08-20T09:28:56Z,肉蛋LowDon,UCELhvN80JBsp3t4g9KZyIHg,https://i.ytimg.com/vi/zga_r5nfErc/hqdefault.jpg
3,社會領域,社會議題,YpWe7FN2Z6M,善良男孩遭老奶奶欺詐：揭露偽裝殘疾的騙局 #社會議題,善良男孩遭老奶奶欺詐：揭露偽裝殘疾的騙局#社會議題.,2025-08-24T09:43:12Z,快樂減速帶,UC2e9A0rc9Ok6hNdT8pJeNUA,https://i.ytimg.com/vi/YpWe7FN2Z6M/hqdefault.jpg
4,社會領域,社會議題,WtPg7uugEBc,現在所有的社會矛盾和難點問題幾乎都是從六七十年代遺留下來的【創史館】#歷史#歷史故事#歷史人...,台灣近代史https://www.youtube.com/playlist?list=PLr...,2025-08-29T10:45:03Z,創史館,UCMYbqjinLC3BlNsP5HPDr3A,https://i.ytimg.com/vi/WtPg7uugEBc/hqdefault.jpg


In [5]:
# ==== 前處理 ====
# Search keyword加入主題建模，內容會過於重複
df_raw['Empty keywords'] = ''
df = preprocess_dataframe(
    df_raw, 
    target_variant="zh-TW",
    merge_fields=["Video title", "Video description", "Empty keywords"]
)
df = df.dropna(subset=["text_translated"]).reset_index(drop=True)
# 此次分析，不想留逗號、句號等有語意的符號
df['text_translated'] = df['text_translated'].apply(remove_all_punctuation)
print(df.head()['text_translated'])

開始合併文字欄位，共 1200 筆資料
開始清理雜訊
開始批次翻譯
偵測所有文本所屬語言
分離目標語言文本和非目標語言文本


批量翻譯非目標語言文本 (共 828 筆):   0%|          | 0/17 [00:00<?, ?it/s]

開始統一中文變體（繁體/簡體）
0    Precht KI vs Sozialsystem 社會保險 五險一金 勞資關係 社會結構 ...
1    現在是時候談論一個禁忌 這是一個社會問題 社會保障 五個月的資金 社會福利 人口利潤 00 ...
2    無退路 不向腐敗低頭 社會議題 臺灣 ai 詞 LowDon 曲 Suno 唱 Suno 錯...
3    善良男孩遭老奶奶欺詐 揭露偽裝殘疾的騙局 社會議題 善良男孩遭老奶奶欺詐 揭露偽裝殘疾的騙局...
4    現在所有的社會矛盾和難點問題幾乎都是從六七十年代遺留下來的 創史館 歷史 歷史故事 歷史人物...
Name: text_translated, dtype: object


In [9]:
embedding_model = SentenceTransformer("shibing624/text2vec-base-chinese")
vectorizer = CountVectorizer(tokenizer=jieba_tokenizer, lowercase=False, min_df=2)   
clusterer = hdbscan.HDBSCAN(
    min_cluster_size=5,   # 一個群集最少需具備樣本數
)

# ==== 依 Topic 分組建模，並產生各自報表 ==== 
grouped_by_topic = df.groupby('Category')
for category_name, group_df in grouped_by_topic: 
    print(f"--- 處理領域: {category_name} ---")
    
    docs = group_df["text_translated"].tolist()
    if not docs:
        print(f"領域 '{category_name}' 沒有資料，跳過。")
        continue

    # ==== 所有圖表共通變數與前置處理 ====
    # 複製df，以避免 SettingWithCopyWarning 錯誤
    group_df = group_df[['Category', 'Search keyword', 'merged_text', 'cleaned_text', 'text_translated']].copy()
    search_keywords = '、'.join(group_df['Search keyword'].unique())

    # 依每圈資料訓練新模型
    topic_model = BERTopic(
        embedding_model=embedding_model,
        language="chinese",
        vectorizer_model=vectorizer,
        hdbscan_model=clusterer,
        verbose=True
    )
    topics, probs = topic_model.fit_transform(docs)
    group_df["topic_id"] = topics
    # 移除未在任何文本出現的領域
    group_df = group_df[group_df['topic_id'] > -1]

    topic_info = topic_model.get_topic_info()    
    topic_info = topic_info[topic_info['Topic'] > -1]

    docs_dir = os.path.join(os.getcwd(), '..', 'docs', category_name)
    os.makedirs(docs_dir, exist_ok=True)
    bubble_path = os.path.join(docs_dir, "topics_bubble.html")
    hier_path = os.path.join(docs_dir, "topics_hierarchy.html")
    sankey_path = os.path.join(docs_dir, "topic_keyword_sankey.html")

    print(len(topic_info))
    if len(topic_info) < 3:
        print('領域數量太少，無法繪圖')

        warning_html = """
        <!DOCTYPE html>
        <html>
        <head>
            <meta charset="UTF-8">
            <title>資料不足</title>
            <style>
                body { font-family: sans-serif; display: flex; justify-content: center; align-items: center; height: 100vh; }
                p { font-size: 1rem; color: gray; }
            </style>
        </head>
        <body>
            <p>該領域分析出來的議題數量過少，無法視覺化。</p>
        </body>
        </html>
        """
        for path in [bubble_path, hier_path, sankey_path]:
            with open(path, "w", encoding="utf-8") as f:
                f.write(warning_html)
        print("Saved:", bubble_path, hier_path, sankey_path)
        
        continue

    # ==== Bubble 圖 ====
    fig_bubble = visualize_topics_bubble(topic_model)
    fig_bubble.update_layout(title="")
    save_plotly_html(fig_bubble, bubble_path)
    
    # 準備html中 議題ID vs 議題內容 對照表資料
    topic_table_df = pd.DataFrame({
        'Topic': topic_info['Topic'],
        'Keywords': topic_info['Name'].apply(lambda x: ' | '.join(x.split('_')[:5]))
    })
    table_data = go.Table(
        header=dict(values=["議題代號", "議題內容"],
                    fill_color='paleturquoise',
                    align='left'),
        cells=dict(values=[topic_table_df.Topic, topic_table_df.Keywords],
                    fill_color='lavender',
                    align='left')
    )
    # 準備html中 Bubble Chart 資料
    bubble_data = fig_bubble.to_json()
    # 整合並寫入新html內容
    html_content = f"""
    <!DOCTYPE html>
    <html>
    <head>
        <meta charset="UTF-8">
        <title>Topic Visualization</title>
        <script src="https://cdn.plot.ly/plotly-latest.min.js"></script>
        <style>
            body {{ font-family: sans-serif; }}
            .container {{ display: flex; width: 100%; height: 80vh; }}
            #bubble-chart {{ flex: 2; border: 1px solid #ccc; box-sizing: border-box; }}
            #topic-table {{ flex: 1; overflow-y: auto; box-sizing: border-box; }}
            .hover-cell {{ cursor: pointer; font-weight: bold; text-decoration: underline; }}
            h3.title {{ margin-block-end: .1em; font-weight: 320; }}
            p.subtitle {{ margin-block-start: 0em; margin-block-end: .4em; font-size: .9rem; font-weight: 300; }}
            p.source {{ font-size: .8rem; font-weight: 300; color: gray; }}
        </style>
    </head>
    <body>
        <h3 class="title">熱門議題聲量與相似度分佈（Bubble Chart）</h3>
        <p class="subtitle">議題氣泡愈大，代表相關影片數量愈多；議題氣泡間愈近，代表相似性愈高。</p>
        <div class="container">
            <div id="bubble-chart"></div>
            <div id="topic-table"></div>
        </div>
        <div><p class="source">{data_source_text(search_keywords)}</p></div>
        <script>
            document.addEventListener('DOMContentLoaded', function() {{
                var bubbleData = {bubble_data};
                var tableData = {table_data.to_json()};
                
                // Fix for topic_id being null
                for (var i = 0; i < bubbleData.data.length; i++) {{
                    if (bubbleData.data[i].topic_id === undefined) {{
                        bubbleData.data[i].topic_id = bubbleData.data[i].name;
                    }}
                }}
    
                Plotly.newPlot('bubble-chart', bubbleData.data, bubbleData.layout);
    
                var tableDiv = document.getElementById('topic-table');
                Plotly.newPlot(tableDiv, [tableData], {{}});
    
                tableDiv.on('plotly_hover', function(data) {{
                    if (data.points.length > 0) {{
                        var hoveredTopicId = data.points[0].cells.values[0][data.points[0].pointIndex];
                        var bubblePlot = document.getElementById('bubble-chart');
                        var traceIndex = bubblePlot.data.findIndex(d => d.topic_id === hoveredTopicId);
                        
                        if (traceIndex !== -1) {{
                            Plotly.restyle(bubblePlot, {{
                                'marker.line.width': [3],
                                'marker.line.color': ['black']
                            }}, [traceIndex]);
                        }}
                    }}
                }});
    
                tableDiv.on('plotly_unhover', function(data) {{
                    if (data.points.length > 0) {{
                        var unhoveredTopicId = data.points[0].cells.values[0][data.points[0].pointIndex];
                        var bubblePlot = document.getElementById('bubble-chart');
                        var traceIndex = bubblePlot.data.findIndex(d => d.topic_id === unhoveredTopicId);
                        
                        if (traceIndex !== -1) {{
                            Plotly.restyle(bubblePlot, {{
                                'marker.line.width': [1],
                                'marker.line.color': ['rgba(0,0,0,0)']
                            }}, [traceIndex]);
                        }}
                    }}
                }});
            }});
        </script>
    </body>
    </html>
    """
    with open(bubble_path, "w", encoding="utf-8") as f:
        f.write(html_content)
        
    # ==== Hierarchy 圖 ====
    fig_h = visualize_hierarchical_clustering(topic_model)
    fig_h.update_layout(
        title="熱門議題層級關聯（Hierarchical Clustering）",
        annotations=[dict(
            text=data_source_text(search_keywords),
            x=0.5, y=-0.15 if len(topic_info)<30 else -0.1,
            xref="paper", yref="paper",
            showarrow=False,
            font=dict(size=12, color="gray")
        )]
    )
    save_plotly_html(fig_h, hier_path)
    
    # ==== Sankey 圖 ====
    # 建立圖中領域格式: "ID_關鍵字1_關鍵字2..."
    topic_label_map = {}
    for index, row in topic_info.iterrows():
        topic_id = row['Topic']
        keywords = '_'.join(row['Name'].split('_')[1:])
        topic_label_map[topic_id] = f"{topic_id}_{keywords}"
    group_df["topic_label"] = group_df["topic_id"].map(topic_label_map)    
    fig_sk = visualize_sankey(
        group_df, 
        topic_column="topic_label", 
        keyword_column="Search keyword",
        topic_prefix="",
    )
    fig_sk.update_layout(
        title=go.layout.Title(
            text='熱門議題與搜尋關鍵字之關聯（Sankey Diagram）<br /><sub>左側為熱門議題，右側為搜尋關鍵字</sub>'
        ),
        annotations=[dict(
            text=data_source_text(search_keywords),
            x=0.5, y=-0.1,
            xref="paper", yref="paper",
            showarrow=False,
            font=dict(size=12, color="gray")
        )],
    )
    save_plotly_html(fig_sk, sankey_path)
    
    # # ==== 儲存模型與標註資料 ====
    # topic_model.save(os.path.join(docs_dir, "bertopic_model"), serialization="safetensors")
    # group_df.to_csv(os.path.join(docs_dir, "docs_with_topics.csv"), index=False, encoding="utf-8-sig")
    
    print("All reports generated successfully!")

2025-09-26 22:18:02,511 - BERTopic - Embedding - Transforming documents to embeddings.


--- 處理領域: 社會領域 ---


Batches:   0%|          | 0/19 [00:00<?, ?it/s]

2025-09-26 22:18:19,680 - BERTopic - Embedding - Completed ✓
2025-09-26 22:18:19,680 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-09-26 22:18:24,433 - BERTopic - Dimensionality - Completed ✓
2025-09-26 22:18:24,451 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-09-26 22:18:24,465 - BERTopic - Cluster - Completed ✓
2025-09-26 22:18:24,465 - BERTopic - Representation - Fine-tuning topics using representation models.
Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\c-116\AppData\Local\Temp\jieba.cache
Loading model cost 0.506 seconds.
Prefix dict has been built successfully.
2025-09-26 22:18:25,137 - BERTopic - Representation - Completed ✓


40


2025-09-26 22:18:27,052 - BERTopic - Embedding - Transforming documents to embeddings.


檔案已成功儲存至: C:\My data\0.change jobs\data_science_practice\etl_showcase\application\..\docs\社會領域\topics_bubble.html
檔案已成功儲存至: C:\My data\0.change jobs\data_science_practice\etl_showcase\application\..\docs\社會領域\topics_hierarchy.html
檔案已成功儲存至: C:\My data\0.change jobs\data_science_practice\etl_showcase\application\..\docs\社會領域\topic_keyword_sankey.html
All reports generated successfully!
--- 處理領域: 科技領域 ---


Batches:   0%|          | 0/19 [00:00<?, ?it/s]

2025-09-26 22:18:42,341 - BERTopic - Embedding - Completed ✓
2025-09-26 22:18:42,342 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-09-26 22:18:42,760 - BERTopic - Dimensionality - Completed ✓
2025-09-26 22:18:42,760 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-09-26 22:18:42,775 - BERTopic - Cluster - Completed ✓
2025-09-26 22:18:42,775 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-09-26 22:18:42,944 - BERTopic - Representation - Completed ✓


30
檔案已成功儲存至: C:\My data\0.change jobs\data_science_practice\etl_showcase\application\..\docs\科技領域\topics_bubble.html
檔案已成功儲存至: C:\My data\0.change jobs\data_science_practice\etl_showcase\application\..\docs\科技領域\topics_hierarchy.html
檔案已成功儲存至: C:\My data\0.change jobs\data_science_practice\etl_showcase\application\..\docs\科技領域\topic_keyword_sankey.html
All reports generated successfully!
